In [1]:
import pandas as pd
import sqlite3
import numpy as np
import plotly.graph_objects as go
from pypfopt import EfficientFrontier, risk_models, expected_returns

In [2]:
con = sqlite3.connect('data/data.db')
df = pd.read_sql("SELECT * FROM PRICES", con=con)
df

,date,symbol,open,high,low,close,volume
0,1970-01-02 00:00:00,MMM,1.82106,1.82804,1.82106,1.82106,1.008334e+05
1,1970-01-05 00:00:00,MMM,1.82106,1.83520,1.82106,1.82804,6.251505e+05
2,1970-01-06 00:00:00,MMM,1.82804,1.84925,1.82804,1.84925,2.464740e+05
3,1970-01-07 00:00:00,MMM,1.84925,1.86358,1.84227,1.85660,2.307936e+05
4,1970-01-08 00:00:00,MMM,1.85660,1.89234,1.85660,1.88509,4.257342e+05
...,...,...,...,...,...,...,...
3433653,2025-02-24 00:00:00,ZTS,162.08000,165.57000,160.82000,165.31000,3.698045e+06
3433654,2025-02-25 00:00:00,ZTS,165.90000,168.99000,163.35000,168.53000,4.003374e+06
3433655,2025-02-26 00:00:00,ZTS,168.31500,169.00000,164.88000,164.99000,2.609923e+06
3433656,2025-02-27 00:00:00,ZTS,165.26500,167.45000,164.50000,164.98000,2.253533e+06


In [ ]:
df.symbol.unique().size

503